In [65]:
#Imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path
%matplotlib inline
import alpaca_trade_api as tradeapi
#import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
from pathlib import Path
import os
import numpy as np
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [66]:
alpaca_api_key = "PK2QATFJUTTJTLNCCC9E"
alpaca_secret_key = "OpvkG0g84P35EjDZlJZf0fMdJ4ksJiY6YwxNrN4d"
print(alpaca_api_key)

PK2QATFJUTTJTLNCCC9E


In [67]:
def calculate_sharpe_ratio(df):
    
    df_pct_change = df.pct_change()
    #daily_return
    sharpe_ratio = df_pct_change['close'].mean() / df_pct_change['close'].std()
    
    #annualized sharpe ratio
    return (252 ** 0.5) * sharpe_ratio

In [68]:
#Establishing test and train data
close_data = BTC['Value'].values
close_data = close_data.reshape((-1,1))

In [69]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Format current date as ISO format
start_date = pd.Timestamp("2011-05-01", tz="America/New_York").isoformat()
today = pd.Timestamp("2021-01-19", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["SPY", "QQQ", "VTI",
           "SPBO", "JNK", "LQD",
           "IYR", "VNQ", "USRT",
           "PSP", "USO", "GLD",
           "MUB", "MUNI", "PZA",
           "VMBS", "MBB", "MBG",
           "TLT", "SCHO", "SHY"
           ]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get closing prices
df_ETFs = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = today
).df

In [70]:
sharpe_ratio_list = []
# for each of the tickers, we will calculate the sharpe ratio and store it in a list of dictionaries 
for ticker in tickers:
    sharpe_ratio = calculate_sharpe_ratio(df_ETFs[ticker])
    sharpe_ratio_list.append({"ticker": ticker, "sharpe_ratio" : sharpe_ratio})

# create a data frame with from the list
df_sharpe_ratio= pd.DataFrame(sharpe_ratio_list)

In [71]:
 #set the tickers
tickers = ["QQQ", 
           "LQD",
           "VNQ",
           "USO", 
           "PZA",
           "VMBS",
           "TLT"
           ]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get closing prices
df_ETFs = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = today
).df
df_ETFs1 = df_ETFs.dropna()

In [97]:
short_window = 50
long_window = 200

df_closing = pd.DataFrame()

df_closing["QQQ"] = df_ETFs1["QQQ"]["close"]
df_closing["LQD"] = df_ETFs1["LQD"]["close"]
df_closing["VNQ"] = df_ETFs1["VNQ"]["close"]
df_closing["USO"] = df_ETFs1["USO"]["close"]
df_closing["PZA"] = df_ETFs1["PZA"]["close"]
df_closing["VMBS"] = df_ETFs1["VMBS"]["close"]
df_closing["TLT"] = df_ETFs1["TLT"]["close"]

df_closing.index = df_closing.index.date
df_closing = df_closing.reset_index()
df_closing = df_closing.rename(columns={'index': 'Date'})
                                   
# Output the data to CSV
#df_closing.to_csv("Resources/Data_ETFs_model_use.csv", encoding='utf-8', index=True)
df_closing

,Date,QQQ,LQD,VNQ,USO,PZA,VMBS,TLT
0,2011-05-02,58.9924,110.31,61.90,44.9395,22.75,50.7700,93.6300
1,2011-05-03,58.6900,110.25,61.39,44.1200,22.82,50.8590,94.1500
2,2011-05-04,58.6000,110.40,61.05,43.2600,22.94,50.8176,94.5900
3,2011-05-05,58.2800,110.79,60.97,39.3280,23.03,50.9880,95.4600
4,2011-05-06,58.4800,110.77,60.45,38.8700,23.00,51.0400,95.0701
...,...,...,...,...,...,...,...,...
2438,2021-01-12,313.9800,135.47,82.10,35.9075,27.04,53.9100,150.9700
2439,2021-01-13,316.0000,136.52,83.17,35.8200,27.09,53.9700,152.6500
2440,2021-01-14,314.2500,135.96,83.65,36.2600,27.05,53.9900,151.2400
2441,2021-01-15,311.9000,136.14,84.63,35.3500,27.10,54.0300,151.8600


In [108]:
# Setting axis and cleaning data
df_closing['Date'] = pd.to_datetime(df_closing['Date'], utc=True)
df_closing.set_axis(df_closing['Date'], inplace=True)

In [109]:
split_percent = 0.80
split = int(split_percent*len(close_data))

In [110]:
close_train = close_data[:split]
close_test = close_data[split:]

In [111]:
date_train = df_closing['Date'][:split]
date_test = df_closing['Date'][split:]
print(len(close_train))
print(len(close_test))

2839
710


In [112]:
look_back = 90

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [113]:
#Build Model
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')





In [114]:
# Show the model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [116]:
# Train the model
num_epochs = 10
model.fit_generator(train_generator, epochs=num_epochs, verbose=0)

ValueError: in user code:

    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
        outputs = m

In [95]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "QQQ",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [96]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = QQQ['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

NameError: name 'QQQ' is not defined

In [ ]:
forecast